# Automatic Number Plate Recognition using Ultralytics YOLO11 + OpenAI `gpt-4o-mini`

This notebook provides a comprehensive guide to implementing automatic number plate recognition (ANPR) using the YOLO11 model in combination with `gpt-4o-mini`.

## What is Automatic Number Plate Recognition (ANPR)?
Automatic number plate recognition is a technology designed to identify and extract vehicle number plate information from images or videos. By leveraging the powerful capabilities of Ultralytics YOLO11 for object detection and OpenAI `gpt-4o-mini` for text recognition, ANPR becomes an efficient solution for automating vehicle identification tasks.

## Why Use YOLO11 + GPT-4o-Mini for ANPR?

- Accurate Detection: YOLO11’s object detection capabilities ensure precise localization of license plates in various conditions, such as low light or high-speed movement.

- Seamless Text Recognition: With `gpt-4o-mini`, extracted license plate regions are processed to recognize alphanumeric text accurately, even with variations in font, angle, or clarity.

- Real-Time Processing: The integration allows for real-time vehicle monitoring, making it ideal for applications in traffic management, parking systems, and security surveillance.🚗

### Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://clickpy.clickhouse.com/dashboard/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
!uv pip install ultralytics

import base64

import cv2
import ultralytics
from openai import OpenAI
from ultralytics import YOLO
from ultralytics.utils.downloads import safe_download
from ultralytics.utils.plotting import Annotator, colors

ultralytics.checks()

Ultralytics 8.3.192  Python-3.13.1 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Setup complete  (32 CPUs, 31.6 GB RAM, 30.7/195.3 GB disk)


### Download the Video and Model File

Download the sample video we’ll use for processing. If you prefer to use your own video file, downloading the sample is not necessary.  

✅ We’ll download a license plate detection model `anpr-demo-model.pt` trained on a small dataset, designed to detect license plates in the sample video. You are welcome to use your own custom models as well.  

⚠️ Note: This license plate detection model `anpr-demo-model.pt` is intended solely for proof of concept (POC) purposes and may only work with `anpr-demo-video.mp4`.

In [2]:
# download the sample video file
safe_download("https://github.com/ultralytics/assets/releases/download/v0.0.0/anpr-demo-video.mp4")

# download the sample model file
safe_download("https://github.com/ultralytics/assets/releases/download/v0.0.0/anpr-demo-model.pt")

WindowsPath('anpr-demo-model.pt')

### Read the Video and Model File

You can either read the video file directly or stream content from an RTSP (Real-Time Streaming Protocol) source, offering flexible video input options to meet your requirements.

✅ By default, we will use the demo video `anpr-demo-video.mp4` downloaded in the previous step. Additionally, we’ll set up the video writer to manage the output video processing.

✅ The model `anpr-demo-model.pt` will also be initialized in memory to handle the processing. You can also use your own model for license plate detection.

In [3]:
cap = cv2.VideoCapture("anpr-demo-video.mp4")
assert cap.isOpened(), "Error reading video file"

# Video writer
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("anpr-output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Load the Ultralytics YOLO license plate detection model
model = YOLO("anpr-demo-model.pt")

<img align="left" src="https://github.com/user-attachments/assets/101152c3-25ba-48a7-9916-c32c3be5857e" height="640">

### Configure OpenAI Client

It’s time to configure the OpenAI client that will accept `base64` image data, extract the number plate text, and return it as a response. The configuration allows you to adjust the content sent to the OpenAI model, and you can also choose different models, such as `gpt-4o`.

⚠️ For the API key, i.e., `OpenAI(api_key="api_key")`, you'll need to visit the [OpenAI API key settings page](https://platform.openai.com/settings/organization/api-keys) and generate an API key for use.

The `extract_text` function is designed to process a base64-encoded image containing a vehicle's number plate.

✅ It sends the image data to the OpenAI client using the `gpt-4o-mini` model and requests the extraction of only the license plate text.

✅ If the text cannot be extracted, the function ensures a response of None. -

✅ Additionally, it filters out any text written near the license plate to provide accurate results.

In [ ]:
client = OpenAI(api_key="")

# Define the text prompt
prompt = """
Can you extract the vehicle number plate text inside the image?
If you are not able to extract text, please respond with None.  # Fallback instruction
Only output text, please.  # Ensure no extra formatting
If any text character is not from the English language, replace it with a dot (.)  # Handle non-English characters, because OpenCV directly can't process these.
"""


def extract_text(base64_encoded_data):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_encoded_data}"},
                    },
                ],
            }
        ],
    )
    return response

### Process Video Frames

In this step, we will process video frames to detect objects, crop regions with padding, and extract license plate text using an OpenAI model. Here's how it works:

✅ Frames are read using OpenCV, and objects are detected using the YOLO11 model.
Bounding boxes are adjusted with padding to crop regions of interest while ensuring proper boundaries.

✅ Cropped regions are encoded in base64 and sent to the `extract_text` function, which uses OpenAI’s model to retrieve license plate text.

✅ The extracted text is added as a label to the bounding box on the video frame.

✅ Processed frames are saved to an output video file.

In [5]:
padding = 10  # Adjust the padding value as needed

while cap.isOpened():
    success, im0 = cap.read()

    if not success:
        break

    results = model.predict(im0)[0].boxes
    boxes = results.xyxy.cpu()
    clss = results.cls.cpu()

    ann = Annotator(im0, line_width=3)

    for (
        cls,
        box,
    ) in zip(clss, boxes):
        height, width, _ = im0.shape  # Get the dimensions of the original image

        # Calculate padded coordinates
        x1 = max(int(box[0]) - padding, 0)
        y1 = max(int(box[1]) - padding, 0)
        x2 = min(int(box[2]) + padding, width)
        y2 = min(int(box[3]) + padding, height)

        # Crop the object with padding and encode the numpy array to base64 format.
        base64_im0 = base64.b64encode(cv2.imencode(".jpg", im0[y1:y2, x1:x2])[1]).decode("utf-8")

        response = extract_text(base64_im0).choices[0].message.content

        print(f"Extracted text: {response}")

        ann.box_label(box, label=str(response), color=colors(cls, True))  # Draw the bounding boxes

    video_writer.write(im0)  # Write the processed video frame

cap.release()  # Release the video capture
video_writer.release()  # Release the video writer


0: 640x384 3 plates, 42.1ms
Speed: 2.5ms preprocess, 42.1ms inference, 55.3ms postprocess per image at shape (1, 3, 640, 384)
Extracted text: 5379MX-4
Extracted text: H0.3MA.6
Extracted text: 8864 KA-4

0: 640x384 3 plates, 17.3ms
Speed: 2.3ms preprocess, 17.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)
Extracted text: 5379MX-4
Extracted text: 3031MA4.
Extracted text: 0064 KA-4

0: 640x384 4 plates, 17.3ms
Speed: 1.8ms preprocess, 17.3ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 384)
Extracted text: 5379MX-4
Extracted text: .JO.1.MA.
Extracted text: 8064 KA.4
Extracted text: None

0: 640x384 4 plates, 17.3ms
Speed: 1.8ms preprocess, 17.3ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 384)
Extracted text: 5379MX-4
Extracted text: 8884 KA.4
Extracted text: 3051MA.
Extracted text: None

0: 640x384 4 plates, 17.2ms
Speed: 1.8ms preprocess, 17.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 384)
Extracted text: 5379MX-

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-tQSDtxvbq4HF6uV9Q01xVu6z on tokens per min (TPM): Limit 200000, Used 200000, Requested 860. Please try again in 258ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

<img align="left" src="https://github.com/user-attachments/assets/d9f5efe7-d058-48f4-9515-f7ccf6084c2f" height="640">